<a href="https://colab.research.google.com/github/maxrgnt/pythdc2-project2/blob/master/cleanBorderData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [186]:
# Panel Data
import pandas as pd
# # System folders
import os
from pathlib import Path

In [189]:
def abbreviate(stateName):
    abrvDict = {'Alaska':'AL',
               'Arizona':'AZ',
               'California':'CA',
               'Idaho':'ID',
               'Maine':'ME',
               'Michigan':'MI',
               'Minnesota':'MN',
               'Montana':'MT',
               'New Mexico':'NM',
               'New York':'NY',
               'North Dakota':'ND',
               'Ohio':'OH',
               'Texas':'TX',
               'Vermont':'VT',
               'Washington':'WA'}
    abrv = ''
    if stateName in abrvDict:
        abrv = abrvDict[stateName]
    return abrv

def safeDrop(df, cols):
    print(df.shape)
    for col in df.columns:
        if col in cols:
            print(f'removing: {col}')
            df.drop([col], axis=1, inplace=True)
    print(df.shape)
    return df

In [190]:
dataPath = Path.joinpath(Path.cwd(),'data','borderCrossing.csv')
df = pd.read_csv(dataPath)
df.sample(3)

,Unnamed: 0,Abrv,State,Longitude,Latitude,Border,Date,Measure,Value
18497,18497,NM,New Mexico,-106.68,31.78,US-Mexico Border,9/1/2014 0:00,Pedestrians,10796
108322,108322,TX,Texas,-99.51,27.50,US-Mexico Border,6/1/1997 0:00,Bus Passengers,25584
53503,53503,CA,California,-116.94,32.55,US-Mexico Border,2/1/2008 0:00,Pedestrians,108995


In [182]:
df['Measure'].value_counts()

Personal Vehicle Passengers    30196
Bus Passengers                 28820
Pedestrians                    28697
Train Passengers               27623
Name: Measure, dtype: int64

In [183]:
# Only interested in Passenger / Pedestrian crossings
people = df['Measure'].str.contains('Passengers|Pedestrians', case = False)

In [184]:
# Check to see how much data frame shrinks after filtering down
print(f'All measures: {df.shape}')
#df = df[people] # This gave index error when re-running, changed to df.loc
df = df.loc[people]
print(f'Just people: {df.shape}')

All measures: (115336, 9)
Just people: (115336, 9)


In [195]:
# Break out Location into latitude and longitude
                # substring from 'POINT( ' to ')' and split on ' ' grabbing first then second element set as float
if 'Location' in df.columns:
    df['Latitude'] = df['Location'].str[len('POINT ('):-1].str.split(' ').str[1].astype(float)
    df['Longitude'] = df['Location'].str[len('POINT ('):-1].str.split(' ').str[0].astype(float)

In [192]:
# Drop unnecessary columns 
df = safeDrop(df, ['Port Code','Port Name','Location','Unnamed: 0','index'])

(115336, 9)
removing: Unnamed: 0
(115336, 8)


In [194]:
# Get state abrv
df['Abrv'] = df['State'].apply(abbreviate)
df.head(3)

,Abrv,State,Longitude,Latitude,Border,Date,Measure,Value
0,NY,New York,-73.44253,44.99001,US-Canada Border,3/1/2019 0:00,Personal Vehicle Passengers,16377
1,ND,North Dakota,-97.24333,48.96639,US-Canada Border,3/1/2019 0:00,Bus Passengers,1054
2,ND,North Dakota,-98.99457,48.94105,US-Canada Border,3/1/2019 0:00,Personal Vehicle Passengers,509


In [49]:
df = df[['Abrv','State','Longitude','Latitude','Border','Date','Measure','Value']]

In [50]:
df.head()

,Abrv,State,Longitude,Latitude,Border,Date,Measure,Value
0,NY,New York,-73.44253,44.99001,US-Canada Border,3/1/2019 0:00,Personal Vehicle Passengers,16377
1,ND,North Dakota,-97.24333,48.96639,US-Canada Border,3/1/2019 0:00,Bus Passengers,1054
2,ND,North Dakota,-98.99457,48.94105,US-Canada Border,3/1/2019 0:00,Personal Vehicle Passengers,509
3,WA,Washington,-122.44316,48.94802,US-Canada Border,3/1/2019 0:00,Pedestrians,79
4,ME,Maine,-67.42955,45.55984,US-Canada Border,3/1/2019 0:00,Pedestrians,3


In [51]:
# Remove non-pedestrian values to shrink file
df.to_csv(Path.joinpath(Path.cwd(),'data','borderCrossing.csv'))